<center>
    <h1 style="color:rgb(40, 0, 77);font-size:40px"><b>NIDS baseado em machine learning - Segurança e Auditoria de Sistemas</b></h1>
</center>

<center><img src="seguranca.jpg" width=800/>

# **Descrição do Problema**

<p>Este Jupyter Notebook será utilizado para solução do problema proposto no projeto da disciplina de Segurança e Auditoria de Sistemas, UTFPR-CP. O objetivo do trabalho é desenvolver um modelo de machine learning que seja capaz de classificar se um fluxo de pacotes em uma rede representa uma ameaça ou não, e, caso represente, qual o tipo específico de ameaça, no caso, um Network Intrusion Detection System (NIDS). O dataset utilizado será o NSL-KDD, disponibilizado pela University of New Brunswick (UNB).</p>

Com isso em mente, ao longo deste notebook serão desenvolvidas as tarefas:
- Análise exploratória, limpeza e manipulação dos dados;
- Criação de gráficos para visualização dos dados;
- Construção de modelos preditivos, buscando testar diferentes algoritmos de classificação.

# **Importação das bibliotecas**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# **Análise Exploratória, limpeza e manipulação dos dados**

In [2]:
#carregando os dados de treino
dados_treino = pd.read_csv("dados/KDDTrain+.txt", header=None)
col_names = ["duration","protocol type","service","flag","src_bytes","dst_bytes",
             "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
             "num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
             "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count",
             "serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
             "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack type","attack level"]
dados_treino.columns = col_names
dados_treino.head(10)

,duration,protocol type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack type,attack level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
6,0,tcp,private,S0,0,0,0,0,0,0,...,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
7,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
9,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21


In [3]:
dados_treino.shape

(125973, 43)

In [10]:
#verificando se há valores nulos
dados_treino.isnull().sum()[dados_treino.isnull().sum() > 0]

Series([], dtype: int64)

In [5]:
#resumo estatístico básico dos atributos
dados_treino.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack level
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,...,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240,19.504060
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,...,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459,2.291503
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,21.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000


In [6]:
#verificando se há valores duplicados
dados_treino.duplicated().sum()

0

In [7]:
#quantidade de registros por tipo de ataque
dados_treino["attack type"].value_counts()

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: attack type, dtype: int64

In [8]:
#dicionario para mapear os tipos de ataque
#0 - normal
#1 - DoS
#2 - Probe
#3 - UR2
#4 - R2L
dic_ataques = {
    "normal": 0,
    "neptune": 1,
    "satan": 2,
    "ipsweep": 2,
    "portsweep": 2,
    "smurf": 1,
    "nmap": 2,
    "back": 1,
    "teardrop": 1,
    "warezclient": 4,
    "pod": 1,
    "guess_passwd": 4,
    "buffer_overflow": 3,
    "warezmaster": 4,
    "land": 1,
    "imap": 4,
    "rootkit": 3,
    "loadmodule": 3,
    "ftp_write": 4,
    "multihop": 4,
    "phf": 4,
    "perl": 3,
    "spy": 4
}

In [9]:
#mapeando os ataques e verificando a contagem
dados_treino["attack type"] = dados_treino["attack type"].map(dic_ataques)
dados_treino["attack type"].value_counts()

0    67343
1    45927
2    11656
4      995
3       52
Name: attack type, dtype: int64